In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Lists to store extracted data
matrix_dimensions = []
mkl_gflops = []
OBLAS_gflops = []


# Read the text file line by line
with open('mkl-output.txt', 'r') as file:
    for line in file:
        # Split line into words
        words = line.strip().split()

        # Extract x value (first number until 'x')
        value = float(words[0].split('x')[0])
        matrix_dimensions.append(value)

        # Extract y value (second-to-last word)
        gflop = words[-2]
        mkl_gflops.append(gflop)


# Read the text file line by line
with open('oblas-output.txt', 'r') as file:
    for line in file:
        # Split line into words
        words = line.strip().split()

        # Extract y value (second-to-last word)
        gflop = words[-2]
        OBLAS_gflops.append(gflop)




FileNotFoundError: [Errno 2] No such file or directory: 'mkl-output.txt'

In [5]:
# Set Seaborn style
sns.set(style='whitegrid')

# Create the plot
plt.figure(figsize=(10, 6))

sns.lineplot(x=matrix_dimensions, y=mkl_gflops, label='MKL')
sns.lineplot(x=matrix_dimensions, y=OBLAS_gflops, label='OBLAS')

# Set labels and title
plt.xlabel('Matrix dimensions')
plt.ylabel('GFLOPs')
plt.title('Comparison between MKL and OBLAS on dgemm.c')

# Add a legend
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()

NameError: name 'x_values' is not defined

<Figure size 1000x600 with 0 Axes>